# Data Analysis: average number of HPD housing complaints citywide vs. in buildings owned by Robert Izsak

Data was downloaded from [NYC Open Data](https://data.cityofnewyork.us/Housing-Development/Housing-Maintenance-Code-Complaints/uwyv-629c) as a .csv file

### The data was filtered for selected years (January 2018 - December 2022) on the NYC Open Data portal. As described by HPD:

"The Department of Housing Preservation and Development (HPD) records complaints that are made by the public through the 311 Citizen Services Center, Code Enforcement Borough Offices or the internet for conditions which violate the New York City Housing Maintenance Code (HMC) or the New York State Multiple Dwelling Law (MDL). Each complaint is associated with one or more problems reported by the complainant. Problems are closed if a tenant verifies by phone that the condition was corrected or an inspection result is entered by an HPD inspector. A complaint is closed when all associated problems are closed."

This analysis, completed using the GroupBy function in Python Pandas, will then be compared to the average number of complaints per building in the 13 buildings owned by landlord Robert Izsak.  

In [3]:
import pandas as pd

In [6]:
dtypes = {
    "ComplaintID":str,
    "BuildingID":str
    
}

citywide_complaint_data = pd.read_csv("../data/Housing_Maintenance_Code_Complaints.csv", dtype=dtypes)

/var/folders/ts/n_0bnjfs37vfxny5hw2lr6_80000gn/T/ipykernel_93152/2431912675.py:7: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  citywide_complaint_data = pd.read_csv("../data/Housing_Maintenance_Code_Complaints.csv", dtype=dtypes)


In [7]:
citywide_complaint_data.head()

,ComplaintID,BuildingID,BoroughID,Borough,HouseNumber,StreetName,Zip,Block,Lot,Apartment,CommunityBoard,ReceivedDate,StatusID,Status,StatusDate
0,8840203,929031,4,QUEENS,129-11,JAMAICA AVENUE,11418.0,9281,44,4D,9,01/01/2018,2,CLOSE,04/10/2018
1,8843534,45664,2,BRONX,2000,ANTHONY AVENUE,10457.0,2814,65,4A,5,01/01/2018,2,CLOSE,04/17/2018
2,8844749,631321,4,QUEENS,6-31,BEACH 9 STREET,11691.0,15584,4,7F,14,01/01/2018,2,CLOSE,04/01/2018
3,8840500,79156,2,BRONX,737,FOX STREET,10455.0,2707,22,3H,2,01/01/2018,2,CLOSE,01/05/2018
4,8841342,6628,1,MANHATTAN,143,AVENUE D,10009.0,379,32,4A,3,01/01/2018,2,CLOSE,01/07/2018


## Used the function .groupby to determine the count of ComplaintID (a unique value for each complaint) associated with each BuildingID (the same ID is assigned to each building)

## Used .describe() to view the average count of ViolationIDs associated with each BuildingID

In [8]:
complaint_tallies = citywide_complaint_data.groupby(
    ["BuildingID"]
    )["ComplaintID"].count(
   
    ).reset_index(
).sort_values(
        by="ComplaintID",
        ascending = False
)

complaint_tallies

,BuildingID,ComplaintID
4617,114412,10768
97216,811407,7259
49152,370489,4320
69550,52288,4140
7561,125695,4139
...,...,...
85007,680122,1
41479,33607,1
41475,336034,1
85010,680135,1


# According to the analysis the cumulative average number of complaints in NYC apartment buildings between 2018 and 2022 was 14.5

## However, there were large outliers - the maximum number of complaints in a single building was 10768, and 50% of buildings contained 2 or fewer violations. 

In [10]:
complaint_tallies.describe()

,ComplaintID
count,118342.000000
mean,14.540341
std,64.818164
min,1.000000
25%,1.000000
50%,2.000000
75%,9.000000
max,10768.000000


# We created a lambda function to filter out of the citywide data for the specific buildings owned by the landlord Robert Izsak, created a new column titled "izsak_buildings.

In [11]:
citywide_complaint_data["izsak_buildings"] = citywide_complaint_data["BuildingID"].apply(
    lambda x: "323187" in str(x).lower() or "74521" in str(x).lower() or "347340" in str(x).lower() or "142778" in str(x).lower() 
    or "192024" in str(x).lower() or "202731" in str(x).lower() or "301247" in str(x).lower() or "391660" in str(x).lower() or "347632" in str(x).lower() or "256023"
    in str(x).lower() or "347311" in str(x).lower() or "187832" in str(x).lower() or "206597"
)

In [12]:
complaint_subset = citywide_complaint_data[
    citywide_complaint_data["izsak_buildings"] == True
    
]

complaint_subset.head()

,ComplaintID,BuildingID,BoroughID,Borough,HouseNumber,StreetName,Zip,Block,Lot,Apartment,CommunityBoard,ReceivedDate,StatusID,Status,StatusDate,izsak_buildings
3913,8843922,142778,3,BROOKLYN,3813,18 AVENUE,11218.0,5417,29,2B,14,01/01/2018,2,CLOSE,01/26/2018,True
7478,8847674,323187,3,BROOKLYN,84,LAWRENCE AVENUE,11230.0,5422,19,B16,14,01/02/2018,2,CLOSE,01/12/2018,True
18260,8859399,142778,3,BROOKLYN,3813,18 AVENUE,11218.0,5417,29,2B,14,01/05/2018,2,CLOSE,01/26/2018,True
20424,8861850,74521,2,BRONX,1701,EASTBURN AVENUE,10457.0,2794,49,2E,4,01/05/2018,2,CLOSE,01/20/2018,True
24574,8866117,142778,3,BROOKLYN,3813,18 AVENUE,11218.0,5417,29,1E,14,01/06/2018,2,CLOSE,01/12/2018,True


In [15]:
complaint_subset_tallies = complaint_subset.groupby(
    ["BuildingID"]
    )["ComplaintID"].count(
    
    ).reset_index(
).sort_values(
    by="ComplaintID", 
        ascending = False
)

In [16]:
complaint_subset_tallies

,BuildingID,ComplaintID
11,74521,135
0,142778,121
3,202731,113
8,347340,86
9,347632,73
4,256023,58
5,301247,49
6,323187,41
2,192024,35
10,391660,32


# The 13 buildings owned by Robert Izsak recorded an average of 65.5 complaints each between 2018 and 2022, compared to the citywide average of 14.5


In [17]:
complaint_subset_tallies.describe()

,ComplaintID
count,12.000000
mean,65.583333
std,39.784552
min,21.000000
25%,34.250000
50%,53.500000
75%,92.750000
max,135.000000
